In [96]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import date

def getDataframe(jsonPath):
  with open(jsonPath) as f:
    data = json.load(f)

  #Create general data frame
  df = pd.DataFrame(columns=["user","likes", "followers", "item0", "item1", "date"])

  #Put the data in the data frame
  for i in range(0,len(data['output'])):
    df.loc[i] = [data['output'][i]["user"], data['output'][i]["likes"], data['output'][i]["followers"], data['output'][i]["item0"], data['output'][i]["item1"],data['output'][i]["date"]]

  #Eliminate rows which no value in likes from json
  df['likes'].replace('', np.nan, inplace=True)
  #df['item0'].replace('N/A N/A', np.nan, inplace=True)
  #print(df)
  df.dropna(subset=['likes'], inplace=True)
  #Se resetean los indices para el siguiente for
  df.reset_index(inplace=True, drop=True)
  
  #Change the type of followers and likes variables (String to int)
  df['followers']=df['followers'].astype(int)
  df['likes']=df['likes'].astype(int)
  df['date']=pd.to_datetime(df['date'])

  
  #Create new colum in panda series format in order to calculate difference in days
  today = pd.to_datetime("today")
  df['diff_days']= (today - df['date']).dt.days + 1
  df['f_score'] = 0.5

  # TODO: Define better formula
  # TODO: Change 36 to 9 and define better day mapping
  df['post_engagement'] = (df['likes'] / df['followers'])*100
  #* (37 - df['diff_days'])

  m0, m1 ,m2, m3, m4, m5 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
  total_avg0, total_avg1, total_avg2, total_avg3, total_avg4, total_avg5 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
  
  df['avg_interval']=0.0


  for i in range(0, len(df['followers'])):

    if(0 <= df['followers'][i] < 5000):
      total_avg0 = df['post_engagement'][i] + total_avg0
      m0 = m0 + 1
    
    elif (5000 <= df['followers'][i] < 20000):
      total_avg1 = df['post_engagement'][i] + total_avg1
      m1 = m1 + 1

    elif (20000 <= df['followers'][i] < 50000):
      total_avg2 = df['post_engagement'][i] + total_avg2
      m2 = m2 + 1

    elif (50000 <= df['followers'][i] < 200000):
      total_avg3 = df['post_engagement'][i] + total_avg3
      m3 = m3 + 1  

    elif (200000 <= df['followers'][i] < 500000):
      total_avg4 = df['post_engagement'][i] + total_avg4
      m4 = m4 + 1 

    elif (df['followers'][i] > 500000):
      total_avg5 = df['post_engagement'][i] + total_avg5
      m5 = m5 + 1

  '''print(m0,m1,m2,m3,m4,m5)
  print(total_avg0,total_avg1,total_avg2,total_avg3,total_avg4,total_avg5)
  print(total_avg0/m0,total_avg1/m1,total_avg2/m2,total_avg3/m3,total_avg4/m4,total_avg5/m5)'''


  #average_engagement per row depending on followers (calculated before avg_eng), we only introduce those values per intervals
  '''OJO CON LO SIGUIENTE (!!AREGLAR!!) --> Si m0,1,2,3=0, error ---> Posible solucion hacer fuera if(m0>1): x=0 else: x=total_avg0/m0 e igual para los otros cn otra letra
    otra posible solución es --> cambiar en el siguiente for el clave0 y hacer la division directamente, es mas costoso? '''
  intervals_average = {"clave0":total_avg0/m0, "clave1":total_avg1/m1, "clave2":total_avg2/m2, "clave3":total_avg3/m3, "clave4":total_avg4/m4,"clave5":total_avg5/m5}
  df['value']=0.0
  j=0

  for i in range(0, len(df['followers'])):
    if((0 <= df['followers'][i] < 5000) and (m0 > 0)):
      df.loc[j,'avg_interval']= intervals_average['clave0']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((5000 <= df['followers'][i] < 20000) and (m1 > 0)):
      df.loc[j,'avg_interval'] = intervals_average['clave1']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((20000 <= df['followers'][i] < 50000) and (m2 > 0)):
      df.loc[j,'avg_interval'] = intervals_average['clave2']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((50000 <= df['followers'][i] < 200000) and (m3 > 0)):
      df.loc[j,'avg_interval'] = intervals_average['clave3']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif((200000 <= df['followers'][i] < 500000) and (m4 > 0)):
      df.loc[j,'avg_interval'] = intervals_average['clave4']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

    elif ((df['followers'][i] > 500000) and (m5 > 0)):
      df.loc[j,'avg_interval'] = intervals_average['clave5']
      df.loc[j,'value']=(df['post_engagement'][i] - df['avg_interval'][i])*0.5
      j=j+1

  
  #Esto era una prueba con unos valores, falta acabar los intervalos, se puede meter en el bucle anterior (ya metido jose)

  '''for i in range(0, len(df['followers'])):
      if((df['followers'][i] >= 0) and (df['followers'][i] < 5000)):
        #lo comparamos con la media del intervalo y ponderamos según nº followers, +followers +peso
        #TODO define intervals weight (0.1 de prueba)
        df['value'][i]=(df['post_engagement'][i] - df['avg_interval'][i])*0.1     

      elif ((df['followers'][i] >= 5000) and (df['followers'][i] < 20000000)):
          df['value'][i]=(df['post_engagement'][i] - df['avg_interval'][i])*0.5'''
    
  return df


def getLabelsDF(df,item):
  """ Parameters: Dataframe, name ofº the column that contains the item. Example 'item0' """
  df_item = pd.DataFrame(columns=["item","counts"])
  j=0
  for i in range(0, len(df[item].value_counts())):
    df_item.loc[j]=df[item].value_counts().index.tolist()[i],df[item].value_counts()[i]
    j=j+1

  return df_item



In [98]:
if __name__ == "__main__":
  postsDF = getDataframe("recognition-output.json")
  topLabelsDF = getLabelsDF(postsDF,'item0')
  bottomLabelsDF = getLabelsDF(postsDF,'item1')

In [100]:

postsDF
topLabelsDF
#bottomLabelsDF

,item,counts
0,N/A N/A,66
1,abrigo_manga_larga negro,34
2,jersey negro,30
3,jersey gris,27
4,abrigo_manga_larga gris,23
5,abrigo_manga_larga marron,19
6,jersey marron,13
7,abrigo_manga_larga verde oscuro,12
8,jersey verde oscuro,12
9,jersey beige,7
